# Using a pretrained model

 Tune a trained model to create an IA that color picture and compare it with the model1

In [1]:
#Depencies
import os
from random import shuffle
from math import floor
from PIL import Image

In [2]:
#Create a CNN
import numpy as np
from keras.callbacks import TensorBoard
from keras.utils import plot_model
from keras.models import load_model, Model
from keras.backend import int_shape

Using TensorFlow backend.


In [3]:
#Data's path
pathColor = "D:/Project/DeepLearning/AlgoPhotoColor/Data/color/"
pathNb = "D:/Project/DeepLearning/AlgoPhotoColor/Data/nb/"

## tune VGG16

### Model creation

In [4]:
#Load vgg16 model, this model was trained on Imagnet dataset
from keras.applications.vgg16 import VGG16

In [5]:
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, MaxPooling2D

In [6]:
vgg16 = VGG16(include_top=False, weights='imagenet' ,input_tensor=None,  pooling=None, input_shape=(200,200,3), classes=1000)

In [7]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [12]:
#Model2 Creation

inputs = Input(name='Model2_Input', shape=(200,200,1)  )#Black and white picture

x = Conv2D(filters=3,kernel_size=3, activation='relu',padding='same')(inputs)

#add vgg16 layers
for lyr in vgg16.layers[:-10]:
    x = lyr(x)
    x.trainable=False


x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=128)(x)


x = Conv2DTranspose(kernel_size=3,activation='relu',strides = 2 , padding='same',filters=3)(x)

model2 = Model(inputs, x, name='model2')


In [13]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Model2_Input (InputLayer)    (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 200, 200, 3)       30        
_________________________________________________________________
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
__________

In [14]:
plot_model(model2,'model2.png')
model2.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

### Loading data

In [ ]:
#Loading train Data
Xtr = []
Ytr = []
file = open("D:/Project/DeepLearning/AlgoPhotoColor/Data/train.txt")
fileTr = file.readlines()
file.close()


for f in fileTr:
    Ytr.append( np.load(pathColor + f[:-1]))
    Xtr.append( np.load(pathNb + f[:-1]))

#Loading test data
Xtes = []
Ytes = []
file = open("D:/Project/DeepLearning/AlgoPhotoColor/Data/test.txt")
fileTes = file.readlines()
file.close()

for f in fileTes:
    Ytes.append( np.load(pathColor + f[:-1]))
    Xtes.append( np.load(pathNb + f[:-1]))

In [ ]:
w , h= (200,200)
# reshape
Xtr = np.asarray(Xtr).flatten()
Xtr = Xtr.reshape(-1,w,h,1)
Ytr = np.asarray(Ytr).flatten()
Ytr = Ytr.reshape((-1,w,h,3))

Xtes = np.asarray(Xtes).flatten()
Xtes = Xtes.reshape(-1,w,h,1)
Ytes = np.asarray(Ytes).flatten()
Ytes = Ytes.reshape((-1,w,h,3))

## Training the model

In [ ]:
epochStart = 0
epochEnd = 30
step = 10

In [ ]:
tbd = TensorBoard("D:/Project/DeepLearning/AlgoPhotoColor/outputs/Log/Model2")

In [ ]:
#Loop through epoch and very step epochs save the model, and genrate some picture

for ep in range(epochStart,epochEnd,step):
    
    #===============Train the model=======================================
    model2.fit(Xtr,Ytr,batch_size=15,callbacks=[tbd],validation_data=(Xtes,Ytes) ,initial_epoch=ep ,epochs= ep+step)
    
    #==================Save the model=========================================
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/models/Model2/Epoch{}'.format(ep + step))#create the folder
    model2.save( "D:/Project/DeepLearning/AlgoPhotoColor/models/Model2/Epoch{}/model2{}.h5".format(ep + step,ep + step))
    
    #==================Create output data=====================================
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}'.format(ep + step))
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/test'.format(ep + step))
    os.mkdir('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/train'.format(ep + step))
    
    
    #export the genrated pictures (test)
    res = model2.predict(Xtes[0:20])
    res *= 255
    res = res.astype('uint8')

    index = 1
    for img in res:
        z = Image.fromarray( img )
        z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/test/Photo{}.png'.format(ep + step,index))
        index += 1
    
    #export the genrated pictures (train)
    res = model2.predict(Xtr[0:20])
    res *= 255
    res = res.astype('uint8')
    index = 1
    for img in res:
        z = Image.fromarray( img )
        z.save('D:/Project/DeepLearning/AlgoPhotoColor/outputs/Model2/Epoch{}/train/Photo{}.png'.format(ep + step,index))
        index += 1
    